#### Flow monitor

In [15]:
import xml.etree.ElementTree as ET
import pandas as pd

# Load the XML file
flowmon_path="/home/nourhen_dev/repos/ns-3-allinone/ns-3.41/flowmon_three_nodes.xml"
tree = ET.parse(flowmon_path)  # Replace with your file path
root = tree.getroot()

# Prepare a list to collect parsed flow data
flow_data = []

# Iterate over all <Flow> elements inside <FlowStats>
#---------------nano seconds to second & bytes to Megabytes
for flow in root.find('FlowStats').findall('Flow'):
    flow_id = int(flow.attrib['flowId'])
    time_first_tx = float(flow.attrib['timeFirstTxPacket'].replace('ns', '').replace('+', ''))*1E-9
    time_first_rx = float(flow.attrib['timeFirstRxPacket'].replace('ns', '').replace('+', ''))*1E-9
    time_last_tx = float(flow.attrib['timeLastTxPacket'].replace('ns', '').replace('+', ''))*1E-9
    time_last_rx = float(flow.attrib['timeLastRxPacket'].replace('ns', '').replace('+', ''))*1E-9
    delay_sum = float(flow.attrib['delaySum'].replace('ns', '').replace('+', ''))*1E-9
    jitter_sum = float(flow.attrib['jitterSum'].replace('ns', '').replace('+', ''))*1E-9
    last_delay = float(flow.attrib['lastDelay'].replace('ns', '').replace('+', ''))*1E-9
    tx_bytes = int(flow.attrib['txBytes'])
    rx_bytes = int(flow.attrib['rxBytes'])
    tx_packets = int(flow.attrib['txPackets'])
    rx_packets = int(flow.attrib['rxPackets'])
    lost_packets = int(flow.attrib['lostPackets'])
    times_forwarded = int(flow.attrib['timesForwarded'])
    duration = time_last_rx - time_first_tx
    throughput_Mbps = (int(flow.attrib['rxBytes']) * 8) / (duration * 1e6)  # bits/sec → Mbps


    flow_data.append({
        'flow_id': flow_id,
        'time_first_tx_s': time_first_tx,
        'time_first_rx_s': time_first_rx,
        'time_last_tx_s': time_last_tx,
        'time_last_rx_s': time_last_rx,
        'delay_sum_s': delay_sum,
        'jitter_sum_s': jitter_sum,
        'last_delay_s': last_delay,
        'tx_bytes': tx_bytes,
        'rx_bytes': rx_bytes,
        'tx_packets_Mb': tx_packets,
        'rx_packets_Mb': rx_packets,
        'lost_packets_Mb': lost_packets,
        'times_forwarded': times_forwarded,
        'duration':duration,
        'throughput_Mbps':throughput_Mbps
    })

classifier_info = {}
for flow in root.find('Ipv4FlowClassifier').findall('Flow'):
    flow_id = int(flow.attrib['flowId'])
    classifier_info[flow_id] = {
        'src': flow.attrib['sourceAddress'],
        'dst': flow.attrib['destinationAddress'],
        'src_port': flow.attrib['sourcePort'],
        'dst_port': flow.attrib['destinationPort'],
        'protocol': flow.attrib['protocol']
    }

# Merge with previous stats if needed
for row in flow_data:
    fid = row['flow_id']
    if fid in classifier_info:
        row.update(classifier_info[fid])
df_flowmon = pd.DataFrame(flow_data)
df_flowmon.head()

,flow_id,time_first_tx_s,time_first_rx_s,time_last_tx_s,time_last_rx_s,delay_sum_s,jitter_sum_s,last_delay_s,tx_bytes,rx_bytes,...,rx_packets_Mb,lost_packets_Mb,times_forwarded,duration,throughput_Mbps,src,dst,src_port,dst_port,protocol
0,1,1.54492,1.54700,8.99989,9.00197,7.41379,0.045420,0.002084,3729340,3729340,...,3545,0,0,7.45705,4.000874,10.1.1.1,10.1.1.2,49154,9,17
1,2,1.61367,1.61575,8.99947,9.00156,10.63850,0.064859,0.002084,5351524,5351524,...,5087,0,0,7.38789,5.794915,10.1.1.1,10.1.1.2,49153,9,17
2,3,1.61545,1.61754,8.79074,8.79283,2.49347,0.016274,0.002084,1253984,1253984,...,1192,0,0,7.17738,1.397707,10.1.1.1,10.1.1.2,49155,9,17
3,4,1.63262,1.63470,8.26971,8.27179,4.49203,0.017217,0.002084,2262852,2262852,...,2151,0,0,6.63917,2.726669,10.1.3.2,10.1.1.2,49154,9,17
4,5,1.85422,1.85630,5.36689,5.36897,6.90462,0.018440,0.002084,3480016,3480016,...,3308,0,0,3.51475,7.920941,10.1.3.2,10.1.1.2,49153,9,17


#### For multiple flows

In [16]:
import pandas as pd

df = df_flowmon

# Create a 'flow_id' column as a tuple of the 5-tuple (src_ip, src_port, dst_ip, dst_port, protocol)
df['flow_id'] = list(zip(df['src'], df['src_port'], df['dst'], df['dst_port'], df['protocol']))

# Alternatively, as a concatenated string (for easier human reading)
# df['flow_id_str'] = df['src'].astype(str) + ':' + df['src_port'].astype(str) + '->' + \
                    # df['dst'].astype(str) + ':' + df['dst_port'].astype(str) + ' [' + \
                    # df['protocol'].astype(str) + ']'


In [21]:
df.head()

,flow_id,time_first_tx_s,time_first_rx_s,time_last_tx_s,time_last_rx_s,delay_sum_s,jitter_sum_s,last_delay_s,tx_bytes,rx_bytes,...,rx_packets_Mb,lost_packets_Mb,times_forwarded,duration,throughput_Mbps,src,dst,src_port,dst_port,protocol
0,"(10.1.1.1, 49154, 10.1.1.2, 9, 17)",1.54492,1.54700,8.99989,9.00197,7.41379,0.045420,0.002084,3729340,3729340,...,3545,0,0,7.45705,4.000874,10.1.1.1,10.1.1.2,49154,9,17
1,"(10.1.1.1, 49153, 10.1.1.2, 9, 17)",1.61367,1.61575,8.99947,9.00156,10.63850,0.064859,0.002084,5351524,5351524,...,5087,0,0,7.38789,5.794915,10.1.1.1,10.1.1.2,49153,9,17
2,"(10.1.1.1, 49155, 10.1.1.2, 9, 17)",1.61545,1.61754,8.79074,8.79283,2.49347,0.016274,0.002084,1253984,1253984,...,1192,0,0,7.17738,1.397707,10.1.1.1,10.1.1.2,49155,9,17
3,"(10.1.3.2, 49154, 10.1.1.2, 9, 17)",1.63262,1.63470,8.26971,8.27179,4.49203,0.017217,0.002084,2262852,2262852,...,2151,0,0,6.63917,2.726669,10.1.3.2,10.1.1.2,49154,9,17
4,"(10.1.3.2, 49153, 10.1.1.2, 9, 17)",1.85422,1.85630,5.36689,5.36897,6.90462,0.018440,0.002084,3480016,3480016,...,3308,0,0,3.51475,7.920941,10.1.3.2,10.1.1.2,49153,9,17


use .nunique() ad .unique() to see the cardinal or the values of the distinct set

In [18]:
num_flows = df['flow_id'].nunique()
print(f"Number of unique flows: {num_flows}")


Number of unique flows: 20


In [31]:
df['src_port'].unique()

array(['49154', '49153', '49155', '49157', '49156', '49158', '49160',
       '49159', '49161'], dtype=object)

In [33]:
df['src'].unique()

array(['10.1.1.1', '10.1.3.2', '10.1.2.1', '10.1.2.2', '10.1.1.2',
       '10.1.3.1'], dtype=object)

In [32]:
df['dst_port'].unique()

array(['9'], dtype=object)